In [1]:
import pandas as pd
from datetime import datetime

In [2]:
'''
En este apartado se realiza la lectura del archivo "datos_arts.csv"
'''
datos_arts = pd.read_csv("datos_arts.csv", parse_dates=["fecha"])
datos_arts.head(3)

,grupo,depto,item,articulo,fecha,precio,unidades,importe,SECTOR
0,15 LIMPIEZA DEL HOGAR,CUIDADO DE LA ROPA,1000025719,1000025719 JABON EN BARRA BULL DOG GLICERINA 2...,2020-08-19,54,7.0,378.0,NO COMESTIBLES
1,15 LIMPIEZA DEL HOGAR,CUIDADO DE LA ROPA,1000025724,1000025724 JABON LIQUIDO PAR D.P. 800 CC NEVEX...,2020-08-19,136,1.0,136.0,NO COMESTIBLES
2,15 LIMPIEZA DEL HOGAR,CUIDADO DE LA ROPA,1000025752,1000025752 JABON LIQ NEVEX MATIC MULTIAC DIF D...,2020-08-19,410,1.0,410.0,NO COMESTIBLES


In [3]:
'''
En este apartado se realiza un Dataframe con las siguientes columnas: Fecha, 
Cantidad_articulos y Total_importe.
'''
df = datos_arts.loc[:,["fecha", "articulo", "importe"]]
df.drop_duplicates(["fecha","articulo"], keep="first")  # Se elimina articulos duplicados por fecha

df = df.groupby("fecha").agg({"articulo": "count",   # Agrupo por fecha
                                 "importe": "sum"})
df.rename(columns={"articulo":"cantidad_articulos",
                   "importe":"total_importe"})

,cantidad_articulos,total_importe
fecha,,
2020-01-09,725,187805.51
2020-01-10,714,217737.20
2020-01-11,769,244827.88
2020-02-09,777,236752.26
2020-02-10,846,280201.06
...,...,...
2020-10-31,852,287103.04
2020-11-09,743,211036.10
2020-11-10,717,194116.77


In [4]:
'''
En este apartado se crea una nueva columna (SECTOR) en el archivo original
cumplimentando las condiciones
'''
sector =[]
def Sector(): # La función recorre la columna grupo y agrega los string correspondientes a una lista
    try:
        for index in range(0, len(datos_arts)):
            if datos_arts["grupo"][index] == "15 LIMPIEZA DEL HOGAR" or datos_arts["grupo"][index] == "18 PERFUMERIA":
                sector.append("NO COMESTIBLES")
            elif datos_arts["grupo"][index] == "4 LACTEOS" or datos_arts["grupo"][index] == "9 FIAMBRERIA":
                sector.append("PERECEDEROS")
            else:
                sector.append(datos_arts["grupo"][index])
        return sector
            
    except Exception as e:
        print(e)

sector = Sector()
datos_arts["SECTOR"] = sector 
datos_arts.to_csv("datos_arts.csv", index=None)
datos_arts.head()

,grupo,depto,item,articulo,fecha,precio,unidades,importe,SECTOR
0,15 LIMPIEZA DEL HOGAR,CUIDADO DE LA ROPA,1000025719,1000025719 JABON EN BARRA BULL DOG GLICERINA 2...,2020-08-19,54,7.0,378.0,NO COMESTIBLES
1,15 LIMPIEZA DEL HOGAR,CUIDADO DE LA ROPA,1000025724,1000025724 JABON LIQUIDO PAR D.P. 800 CC NEVEX...,2020-08-19,136,1.0,136.0,NO COMESTIBLES
2,15 LIMPIEZA DEL HOGAR,CUIDADO DE LA ROPA,1000025752,1000025752 JABON LIQ NEVEX MATIC MULTIAC DIF D...,2020-08-19,410,1.0,410.0,NO COMESTIBLES
3,15 LIMPIEZA DEL HOGAR,CUIDADO DE SUPERFICIES,1000025907,1000025907 PANO DE PISO STANDARD 50X55 CM,2020-08-19,79,1.0,79.0,NO COMESTIBLES
4,15 LIMPIEZA DEL HOGAR,CUIDADO DE SUPERFICIES,1000025908,1000025908 REJILLA 37X37CM.,2020-08-19,59,1.0,59.0,NO COMESTIBLES


In [5]:
'''
En este apartado se obtiene un df bajo el siguiente formato:
    * agregar dos columnas correspondientes a promedio de precio y suma de importe por cada sector.
    * Agrupado por mes.
'''
datos_arts["fecha"] = pd.to_datetime(datos_arts["fecha"], format="%b-%y") # Convierto a la columna en el formato correcto
datos_arts["fecha"] = datos_arts["fecha"].dt.strftime("%b-%y")

df1 = datos_arts.loc[:,["fecha", "SECTOR", "precio", "importe"]].groupby(["fecha", "SECTOR"]).agg({"precio": "mean",
                                                                                                   "importe": "sum"})
df1.rename(columns={"precio": "prom_precio",
                    "importe": "sum_importe"}, inplace=True)

columna = ["BEBIDAS prom_precio",   # Nombres de columnas para df final 
           "COMESTIBLES prom_precio",
           "NO COMESTIBLE prom_precio",
           "PERECEDEROS prom_precio",
           "BEBIDAS sum_importe", 
           "COMESTIBLES sum_importe",
           "NO COMESTIBLE sum_importe",
           "PERECEDEROS sum_importe"
           ]

pivot = pd.pivot_table(data=df1, index="fecha", values=["prom_precio", "sum_importe"], columns="SECTOR")   # Cambio el indice, columna y los valores.
pivot = pivot.droplevel(level=0, axis=1)
pivot.set_axis([columna], axis=1, inplace=True)

columnas_ordenadas = ["COMESTIBLES prom_precio", 
                      "COMESTIBLES sum_importe", 
                      "BEBIDAS prom_precio",
                      "BEBIDAS sum_importe",
                      "NO COMESTIBLE prom_precio",
                      "NO COMESTIBLE sum_importe",
                      "PERECEDEROS prom_precio",
                      "PERECEDEROS sum_importe"]
pivot = pivot[columnas_ordenadas]
pivot

/tmp/ipykernel_24557/3731666819.py:26: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  pivot.set_axis([columna], axis=1, inplace=True)


,COMESTIBLES prom_precio,COMESTIBLES sum_importe,BEBIDAS prom_precio,BEBIDAS sum_importe,NO COMESTIBLE prom_precio,NO COMESTIBLE sum_importe,PERECEDEROS prom_precio,PERECEDEROS sum_importe
fecha,,,,,,,,
Apr-20,97.929078,407681.0,86.798429,116546.0,176.588339,170881.0,140.013761,60706.81
Aug-20,92.080411,2096150.0,83.382908,521907.0,160.118834,1029138.0,135.183186,292952.78
Dec-20,95.763126,256723.0,80.770925,68732.0,166.905149,104051.0,136.584000,35895.34
Feb-20,97.418951,418015.0,79.379888,101480.0,169.276740,182357.0,137.122549,55653.68
Jan-20,93.456747,354155.0,84.492837,91917.0,168.264479,151680.0,144.318919,52618.59
Jul-20,94.025101,391661.0,83.505181,101851.0,170.331230,182948.0,135.292035,63208.65
Jun-20,97.490741,395455.0,80.782857,91402.0,168.850719,154095.0,143.466346,57350.27
Mar-20,98.531250,449992.0,86.860724,104432.0,168.237785,190594.0,138.167442,64994.98
May-20,99.348165,433864.0,92.489011,110788.0,171.137705,179227.0,124.700000,59489.57
